## Lecture 15

In [ ]:
my_var = 10

In [9]:
print(my_var)

10


In [10]:
print(id(my_var))

4316377440


In [11]:
print(hex(id(my_var)))

0x10146b160


## Lecture 16

In [4]:
import sys

In [5]:
a = [1, 2, 3]

In [6]:
id(a)

4572661960

In [7]:
sys.getrefcount(a)

2

In [8]:
import ctypes

In [11]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value


In [12]:
ref_count(id(a))

1

In [13]:
b = a

In [14]:
ref_count(id(a))

2

In [15]:
ref_count(id(b))

2

## Lecture 17

In [17]:
import ctypes
import gc

In [18]:
def ref_count(address: int):
    return ctypes.c_long.from_address(address).value

In [19]:
def object_by_id(object_id):
    for obj in gc.get_objects():
        if id(obj) == object_id:
            return "Object exists"
    return "Not Found"

In [20]:
class A:
    def __init__(self):
        self.b = B(self)
        print('A: self: {0}, b: {1}'.format(hex(id(self)), hex(id(self.b))))

In [21]:
class B:
    def __init__(self, a):
        self.a = a
        print('B: self: {0}, a: {1}'.format(hex(id(self)), hex(id(self.a))))

In [22]:
gc.disable()

In [23]:
my_var = A()

B: self: 0x11091eb00, a: 0x11091eac8
A: self: 0x11091eac8, b: 0x11091eb00


In [26]:
a_id = id(my_var)
b_id = id(my_var.b)

In [27]:
print(hex(a_id))
print(hex(b_id))

0x11091eac8
0x11091eb00


In [28]:
ref_count(a_id)

2

In [29]:
ref_count(b_id)

1

In [30]:
object_by_id(a_id)

'Object exists'

In [31]:
object_by_id(b_id)

'Object exists'

In [32]:
my_var = None

In [33]:
ref_count(a_id)

1

In [34]:
ref_count(b_id)

1

In [35]:
object_by_id(a_id)

'Object exists'

In [36]:
object_by_id(b_id)

'Object exists'

In [37]:
gc.collect()

673

In [38]:
object_by_id(a_id)

'Not Found'

In [39]:
object_by_id(b_id)

'Not Found'

In [41]:
gc.enable()

## Lecture 20

In [43]:
my_list = [1, 2, 3]

In [44]:
id(my_list)

4572955976

In [45]:
my_list = my_list + [4]

In [46]:
id(my_list)

4572663176

## Lecture 24

In [1]:
c = int()

In [2]:
c

0

In [3]:
c = int('101', base=2)

In [4]:
c

5

In [5]:
def square(a):
    return a ** 2

In [6]:
type(square)

function

In [7]:
print(type(square))

<class 'function'>


## Lecture 25

**Interning** -- reusing objects on demand

At startup CPython pre-loads (caches) a global list of integers in the range **(-5, 256)**

In [9]:
a = 256
b = 256
a is b

True

In [10]:
a = 257
b = 257
a is b

False

## Lecture 26

In [11]:
a = 'hello'
b = 'hello'

In [18]:
print(id(a), id(b))

4339957232 4339742960


In [19]:
a = 'hello world'
b = 'hello world'

In [20]:
print(id(a), id(b))

4339956848 4339955056


In [21]:
a == b

True

In [22]:
a is b

False

In [23]:
a = '_this_is_a_long_string_that_could_be_used_as_an_ientifier'

In [24]:
b = '_this_is_a_long_string_that_could_be_used_as_an_ientifier'

In [25]:
a is b

True

**interned because it could be identifer**

In [26]:
import sys

In [27]:
a = sys.intern('hello world')

In [28]:
b = sys.intern('hello world')

In [29]:
c = 'hello world'

In [30]:
print(id(a), id(b), id(c))

4340327792 4340327792 4340326768


In [31]:
a == b

True

In [32]:
a is b

True

## Lecture 27

### Compile time optimizations:

 - numeric calculations  
 **_(24 * 60)_**
 - short sequences length < 20  
 **_(1,2) * 5_**
 
 **_'abc' * 3_**
 
 **_'hello' + ' world'_**
 - membership tests
 
 **_if e in [1, 2, 3] will be repaced by tuple, sets -> frozen_sets_**

In [38]:
def my_func():
    a = 24 * 60
    b = (1, 2) * 5
    c = 'abc' * 3
    d = 'ab' * 11
    e = 'the quick brown fox' * 7
    f = ['a', 'b'] * 3

In [39]:
my_func.__code__.co_consts

(None,
 1440,
 (1, 2, 1, 2, 1, 2, 1, 2, 1, 2),
 'abcabcabc',
 'ababababababababababab',
 'the quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown foxthe quick brown fox',
 'a',
 'b',
 3)

In [40]:
def my_func(e):
    if e in [1, 2, 3]:
        pass


In [41]:
my_func.__code__.co_consts

(None, (1, 2, 3))

In [42]:
def my_func(e):
    if e in {1, 2, 3}:
        pass


In [43]:
my_func.__code__.co_consts

(None, frozenset({1, 2, 3}))